<div> 
<img src="https://res-5.cloudinary.com/crunchbase-production/image/upload/c_lpad,h_256,w_256,f_auto,q_auto:eco/v1455514364/pim02bzqvgz0hibsra41.png" align="right"><br><br><FONT FACE="times new roman" SIZE=5>
<b>Parcial 2 - Punto 1 </b>
<br>
<i><b>Estudiantes:</b></i> 
<br>
Miguel Angel Rojas, Luis Felipe velasquez
<br>
<i><b>Docentes:</b></i> 
<br>
Camilo Rodríguez
<br>
<i><b>Asignatura:</b></i> 
<br>
Inteligencia Artificial

</FONT>
</div>

# **1. Importación de librerias**

In [ ]:
from IPython.display import display
from copy import copy,deepcopy
import ipywidgets as widgets
import time
import random

## **2. Clase tablero**

In [ ]:
class Tablero:
  #Contstructor de la clase
  def __init__(self, tamanoCelda=(40, 40), nCeldas=None,ocupados=[],coordenadas=[]):
    #Definición de los atributos
    if nCeldas==None:
      self.nCeldas = (3,3)
    else:
      self.nCeldas = nCeldas
    self.out = widgets.HTML()
    display(self.out)
    self.tamanoCelda = tamanoCelda
    self.ocupados=ocupados
    self.coordenadas=[]

  #Metodo que representa graficamente el tablero
  def dibujar(self, objetos=[]):
    tablero = "<table border='1'>{}</table>"
    filas = ""
    for i in range(self.nCeldas[1]):
      s = ""              
      for j in range(self.nCeldas[0]):
        contenido =""
        for o in objetos:
          if o.x == j and o.y == i:
            contenido = \
            "<div style='transform: rotate({angulo}deg);font-size:{tamanoEmoticon}px;'>{emoticon}</div>".\
            format(angulo=o.angulo,tamanoEmoticon=o.tamanoEmoticon, emoticon=o.emoticon)     
        s += "<td style='height:{alto}px;width:{ancho}px;'>{contenido}</td>".\
          format(alto=self.tamanoCelda[0], ancho=self.tamanoCelda[1], 
                contenido=contenido)
      filas += "<tr>{}</tr>".format(s)
    tablero = tablero.format(filas)
    self.out.value=tablero


## **3. Clase Agente_numero**

In [ ]:
#La clase Agente_numero funciona para la graficación del problema
class Agente_numero:
  def __init__(self,numero=None, x=0, y=0, angulo=0, emoticon="❌ ", tamanoEmoticon=30,tablero=Tablero()):
    self.numero=numero
    self.x=x
    self.y=y
    self.angulo=angulo
    self.emoticon=emoticon
    self.tamanoEmoticon=tamanoEmoticon

  #Con este mtodo, se mueve el objeto en posiciones x y y dentro del tablero
  def moverNumero(self,x,y):
    self.x=x
    self.y=y

HTML(value='')

## **4. Clase estado**

In [ ]:
#La clase estado representa un estado en el que puede estar el problema del pluzee
class estado:
    def __init__(self,array=[],origen=None):
      self.array=array #Contiene el arreglo del estado, por ejemplo [3,None,4],[1,8,5],[7,2,6]
      self.origen=origen  #Contiene el objeto del cual probiene este etado(subindice)

    # Muestro del estado
    def __str__(self):
      return f'{self.array[0]}\n {self.array[1]} \n {self.array[2]}\n'

    #Validación de igualdad para dos objetos de esta clase
    def __eq__(self, otro):
      return self.array==otro.array
    
    def __repr__(self):
      return self.__str__()

    #En el siguiente metodo, se obtiene un arreglo de objetos, donde se toma como referencia un estado para sacar sus posibles estados siguentes
    def getEstadosPosibles(self):
        #declaración de variables
        x=0;y=0;estados=[];objEstados=[]
        estado_actual=self.array
        #Cliclo para el numero de celdas del juego, el cual es 3
        for i in range(3):
          for j in range(3):
            if estado_actual[i][j] == None: # Se encuentra la posición None, la cual es la vacia donde los otros numeros cercanos mueven mover
              x=i;y=j
        #A continuacion, se obtienen los estados posibles de acuerdo a la posicion del None, con el fin de que no se saliera del limite
        if x>0:
          nuevo=deepcopy(estado_actual) #Copia del estado base
          #Se intercambian las posiones del None y el numero
          nuevo[x][y]=nuevo[x-1][y]
          nuevo[x-1][y]=None
          estados.append(nuevo)#Se agrega al arreglo de estados
        if x<2:
          nuevo=deepcopy(estado_actual) #Copia del estado base
          #Se intercambian las posiones del None y el numero
          nuevo[x][y]=nuevo[x+1][y]
          nuevo[x+1][y]=None
          estados.append(nuevo)

        if y>0:
          nuevo=deepcopy(estado_actual) #Copia del estado base
          #Se intercambian las posiones del None y el numero
          nuevo[x][y]=nuevo[x][y-1]
          nuevo[x][y-1]=None
          estados.append(nuevo)#Se agrega al arreglo de estados
        if y<2:
          nuevo=deepcopy(estado_actual) #Copia del estado base
          #Se intercambian las posiones del None y el numero
          nuevo[x][y]=nuevo[x][y+1]
          nuevo[x][y+1]=None
          estados.append(nuevo)#Se agrega al arreglo de estados
        #Se crea el arreglo de objetos en base a los estados obtenidos y el origen es self porque de este con respecto a este estado se obtuvo
        for i in estados: objEstados.append(estado(array=i,origen=self))
        return objEstados #Retorno del arreglo de objetos

In [ ]:
coordenadas=[]
def get_randomxy():
    while (True):
      cor=(random.randint(0,2),random.randint(0,2))
      if not cor in coordenadas :
        break
    coordenadas.append(cor)
    return cor

## **5. Metodo Obtenersolucion**

In [ ]:
#Con este metodo, luego de hacer el algoritmo de BFS, se puede obtener la solucion mas optima para solucionar el juego
def obtenerSolucion(visitados): #Como argumento esta los visitados para conocer el orden que se desarrollo
  solucion=[visitados[-1]] #ultima posición
  a=len(visitados) #Tamaño del array visitados
  for i in reversed(visitados): # Se itera en reversa para conocer los origenes de cada estado
    a-=1
    ante=solucion[-1].origen.array #Origen del estado a evaluar
    if a!=0:
      if i.array==ante: # si el origen de estado es igual al que se esta iterando, se agrega al array de solucion
        solucion.append(i)
  return solucion[::-1] #Se retorna el array de solucion al reves

## **6. Metodo CambiarEstadotablero**

In [ ]:
#Con esta funcion, se cambian las posiciones de los numeros en el tablero en base a los estados
def cambiarEstadoTablero(numeros,estado_actual):
  for i in range(3):#el rango es 3 porque el juego es 3x3
    for j in range(3):
        estado=estado_actual.array[i][j] #Se obtiene el numero en cierta posicion
        num=list(filter(lambda i: i.numero == estado , numeros)) #Filtrado par conocer el objeto corresppondient al numero de la posicion validada
        if estado!=None:
          num[0].moverNumero(j,i)#Se mueve el numero en el tablero

In [ ]:
# Estado Inicial
# estadoIncial= [[None,None,None],[None,None,None],[None,None,None]]
# for i in range(8):
#     ran=get_randomxy()
#     estadoIncial[ran[0]][ran[1]]=i+1

## **7. Estados Iniciales**

In [ ]:
#Se tomo un arreglo de diferentes configuraciones iniciales para el juego, 
#debido a que si se colocaba random,algunas de estos estados iniciales podian consumir gran parte de la memoria de la maquina
estadosIniciales=[[[3,None,4],[1,8,5],[7,2,6]] ,[[3,None,4],[1,8,5],[7,2,6]],[[8,1,4],[7,None,3],[2,6,5]] ] #Arreglo de algunas de las configuraciones iniciales
arrayIncial=estadosIniciales[random.randint(0,len(estadosIniciales)-1)] # Se elige una configuracion random del arreglo anterior
arrayFinal=[[1,2,3],[8,None,4],[7,6,5]] # Arreglo que tiene el estado objetivo, es decir el final del juego

In [ ]:
#Se puede apreciar el estado inicial 
arrayIncial

[[8, 1, 4], [7, None, 3], [2, 6, 5]]

## **8. Busqueda en anchura(BFS)**

In [ ]:
# A continuación se desarrolla la implentacion del aloritmo BFS acorde al juego pluze
estadoActual=estado(array=arrayIncial) #Se crea el objeto estado correspondinet al inicial
estadoFinal=estado(array=arrayFinal)#Se crea el objeto estado correspondinet al final u objetivo
cola=[estadoActual] # el array cola tendra los estados posibles para cada configracion, se inicializa con el actual porque es el primero a atender
visitados=[] # Tendra los estados  por los que ya se paso y que son diferentes al estado final
#Ciclo que se repite mientras el estado actual es diferente al objetivo
while estadoActual !=estadoFinal:
  estadoActual=cola.pop(0) # Se elimina la primera poicion de la cola 
  if not estadoActual in visitados: # Si es estado actual no esta en visitados, se agrega el estado a visitados para que no vuelva pasar por este
    visitados.append(estadoActual)
  nuevosEstados=estadoActual.getEstadosPosibles() #Se generan todos los estados posibles de acuerdo al actual
  for i in nuevosEstados: #Se iteran los estados posibles
    if not i in visitados:#Si estso no estan en visitados se agregan a la cola para que sean atendidos
      cola.append(i)
print(estadoActual.array) #Impresion del estado Actual luego de la ejecucion

[[1, 2, 3], [8, None, 4], [7, 6, 5]]


## **9. Graficación de la solución**

In [ ]:
escenario=Tablero() #Creación del objeto tablero
emoticonNumeros=['1️⃣','2️⃣','3️⃣','4️⃣','5️⃣','6️⃣','7️⃣','8️⃣'] #Emojis a utilizar en el tablero
numeros=[]
aux=0 
#a el array numeros, se agregar los objetos numeros para la graficacion:
for i in emoticonNumeros:
    aux+=1
    numeros.append(Agente_numero(numero=aux,tablero=escenario,emoticon=i))

sol=obtenerSolucion(visitados) #Se obtiene la solucion optima del juego 
for i in sol:#Iteración de el arreglo solucion
  cambiarEstadoTablero(numeros,i) #Por cada estado de la solucion se cambia el estad del tablero
  escenario.dibujar(objetos=numeros)#Por cada estado de la solucion se cambia el dibuja el tablero
  time.sleep(1)

HTML(value='')